# Agent Based Model for Bird Flock Problem (2-D Version)

## Set up Boid Agent

In [1]:
class Boid:
  def __init__(self):
    self.x = 0 # x-position
    self.y = 0 # y-position
    self.vx = 0 # x-velocity
    self.vy = 0 # y-velocity

boids = [Boid() for i in range(100)] # Create 100 boids

## Set up the Parameters

In [2]:
# Static Parameters
turnFactor = 0.01  # How much the boids can turn (Default: 0.01, static)
visualRange = 40  # How far the boids can see (Default: 40, static)
protectionRange = 8  # How far the boids can keep away from each other (Default: 8, static)
centeringFactor = 0.0005  # How much the boids move towards the center (Default: 0.0005, static)
avoidFactor = 0.05  # How much the boids avoid each other (Default: 0.05, static)
matchingFactor = 0.05  # How much the boids match each other's speed (Default: 0.05, static)
maxSpeed, minSpeed = 6, 3  # Maximum and minimum speed of the boids (Default: 6, 3, static)
maxBias = 0.01  # Maximum bias value for the boids to move towards the center (Default: 0.01, static)
biasIncrement = 0.00004  # How much the bias value increases each time (Default: 0.00004, static)
# Dynamic Parameters
biasValue = 0.001  # Bias value for the boids to move towards the center (Default: 0.001, changable)

## Set up Boid Behaviors (Functions)

#### Helper Functions

In [6]:
def distance(boid1, boid2):
  """
    Calculate the distance between two boids
  :param boid1: the first boid
  :param boid2: the second boid
  :return: the distance between the two boids
  """
  return ((boid1.x - boid2.x)**2 + (boid1.y - boid2.y)**2)**0.5

#### Separation

In [7]:
def separation(boid, boids):
  """
    Calculate the velocity of the boid to avoid other boids
  :param boid: the boid to calculate the velocity
  :param boids: all the boids
  :return: the velocity of the boid (x-velocity, y-velocity)
  """
  closeDistanceX, closeDistanceY = 0, 0
  numBoids = 0
  for otherBoid in boids:
    if otherBoid != boid and distance(boid, otherBoid) < protectionRange:
        closeDistanceX += boid.x - otherBoid.x
        closeDistanceY += boid.y - otherBoid.y
        numBoids += 1
  if numBoids > 0:
    boid.vx = closeDistanceX * avoidFactor
    boid.vy = closeDistanceY * avoidFactor
  return boid.vx, boid.vy

#### Alignment

In [8]:
def alignment(boid, boids):
  """
    Calculate the velocity of the boid to match the velocity of other boids
  :param boid: the boid to calculate the velocity
  :param boids: all the boids
  :return: the velocity of the boid (x-velocity, y-velocity)
  """
  avgVx, avgVy = 0, 0
  numBoids = 0
  for otherBoid in boids:
    if otherBoid != boid and distance(boid, otherBoid) < visualRange:
      avgVx += otherBoid.vx
      avgVy += otherBoid.vy
      numBoids += 1
  if numBoids > 0:
    avgVx /= numBoids
    avgVy /= numBoids
    boid.vx += (avgVx - boid.vx) * matchingFactor
    boid.vy += (avgVy - boid.vy) * matchingFactor
  return boid.vx, boid.vy

#### Cohesion

In [9]:
def cohension(boid, boids):
  """
    Calculate the velocity of the boid to move towards the center of other boids
  :param boid: the boid to calculate the velocity
  :param boids: all the boids
  :return: the velocity of the boid (x-velocity, y-velocity)
  """
  centeringX, centeringY = 0, 0
  numBoids = 0
  for otherBoid in boids:
    if otherBoid != boid and distance(boid, otherBoid) < visualRange:
      centeringX += otherBoid.x
      centeringY += otherBoid.y
      numBoids += 1
  if numBoids > 0:
    centeringX /= numBoids
    centeringY /= numBoids
    boid.vx += (centeringX - boid.x) * centeringFactor
    boid.vy += (centeringY - boid.y) * centeringFactor
  return boid.vx, boid.vy

#### Edge Turning

In [10]:
def edgeTurning(boid, leftMargin, rightMargin, bottomMargin, topMargin):
  """
    Calculate the velocity of the boid to turn away from the edge of the screen
  :param boid: the boid to calculate the velocity
  :return: the velocity of the boid (x-velocity, y-velocity)
  """
  if boid.x < leftMargin:
    boid.vx += turnFactor
  elif boid.x > rightMargin:
    boid.vx -= turnFactor
  if boid.y < bottomMargin:
    boid.vy += turnFactor
  elif boid.y > topMargin:
    boid.vy -= turnFactor
  return boid.vx, boid.vy

#### Bias

In [11]:
def bias(boid):
    """
        Calculate the velocity of the boid to move towards the center of the screen
    :param boid: the boid to calculate the velocity
    :return: the velocity of the boid (x-velocity, y-velocity)
    """
    boid.vx += (1-biasValue) * boid.vx + biasValue
    boid.vy += (1-biasValue) * boid.vy + biasValue
    #TODO: we can make the bias value change over time
    return boid.vx, boid.vy

#### Speed Limit

In [12]:
def speedLimit(boid):
  """
    Calculate the velocity of the boid to limit the speed
  :param boid: the boid to calculate the velocity
  :return: the velocity of the boid (x-velocity, y-velocity)
  """
  speed = (boid.vx**2 + boid.vy**2)**0.5
  if speed > maxSpeed:
    boid.vx *= maxSpeed / speed
    boid.vy *= maxSpeed / speed
  elif speed < minSpeed:
    boid.vx *= minSpeed / speed
    boid.vy *= minSpeed / speed
  return boid.vx, boid.vy

#### Update Boid Velocity and Position

In [13]:
def updateBoid(boid, boids, leftMargin, rightMargin, bottomMargin, topMargin):
  """
    Update the velocity and position of the boid
  :param boid: the boid to update
  :param boids: all the boids
  :return: the updated boid
  """
  boid.vx, boid.vy = separation(boid, boids)
  boid.vx, boid.vy = alignment(boid, boids)
  boid.vx, boid.vy = cohension(boid, boids)
  boid.vx, boid.vy = edgeTurning(boid, leftMargin, rightMargin, bottomMargin, topMargin)
  boid.vx, boid.vy = bias(boid)
  boid.vx, boid.vy = speedLimit(boid)
  boid.x += boid.vx
  boid.y += boid.vy
  return boid

## Set up Visualisation

In [23]:
import numpy as np
from numpy import random
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pg

app = pg.mkQApp("Plotting Example")
pg.plot(np.random.normal(size=100), title="Simplest possible plotting example")
pg.exec()


0